*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


### Imports

In [1]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.8.0


### Downloading

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2022-04-19 13:35:22--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K  --.-KB/s    in 0.03s   

2022-04-19 13:35:22 (10.5 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2022-04-19 13:35:22--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[==========

In [3]:
df_train = pd.read_table(train_file_path, header=0, names=['indicates', 'text'], usecols=['indicates', 'text'])
df_test = pd.read_table(test_file_path, header=0, names=['indicates', 'text'], usecols=['indicates', 'text'])

#### Processing data

##### Transforming of categorical labels

In [4]:
# Changing categorical labels to numbers (0,1)
df_train['indicates'] = df_train['indicates'].replace("ham", 0)
df_train['indicates'] = df_train['indicates'].replace("spam", 1)
df_test['indicates'] = df_test['indicates'].replace("ham", 0)
df_test['indicates'] = df_test['indicates'].replace("spam", 1)

##### Finding the unique vocabs in the train and test dataset

In [5]:
# making the dataset in the type of "tf.data.Dataset"
train_data = tf.data.Dataset.from_tensor_slices((df_train['text'].values, df_train['indicates'].values))
test_data = tf.data.Dataset.from_tensor_slices((df_test['text'].values, df_test['indicates'].values))

In [14]:
# Defining a tokenizer
tokenizer = tfds.deprecated.text.Tokenizer()

In [ ]:
# Creating vocabulary dataset
vocab_set = set()
# Getting the unique vocabs from train and test data
for text, _ in train_data.concatenate(test_data):
  tokens = tokenizer.tokenize(text.numpy())
  vocab_set.update(tokens)

vocab_count = len(vocab_set)

##### Encoding the text

In [10]:
# Defining an encoder to encode the vocabs 
encoder = tfds.deprecated.text.TokenTextEncoder(vocab_set)

In [11]:
# Defining an encoding function
def encode(text, label):
  encoded_text = encoder.encode(text.numpy())
  return encoded_text, label

In [12]:
def map_fn(text, label):
  encoded_text, label = tf.py_function(encode, inp=[text, label], Tout=(tf.int64, tf.int64))

  encoded_text.set_shape([None])
  label.set_shape([])

  return encoded_text, label

In [13]:
train_data_encoded = train_data.map(map_fn)
test_data_encoded = test_data.map(map_fn)

##### Padding (& shuffling) dataset

In [18]:
BUFFER_SIZE = 1000
BATCH_SIZE = 32
train_dataset = (train_data_encoded.shuffle(BUFFER_SIZE).padded_batch(BATCH_SIZE))
test_dataset = (test_data_encoded.padded_batch(BATCH_SIZE))

### Defining, compiling and training the model

In [19]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(encoder.vocab_size, 32),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [20]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), optimizer='adam', metrics=['accuracy'])

In [21]:
history = model.fit(train_dataset, epochs=10, validation_data=test_dataset, validation_steps=30)

Epoch 1/10
131/131 [==============================] - 17s 56ms/step - loss: 0.2703 - accuracy: 0.9062 - val_loss: 0.0640 - val_accuracy: 0.9781
Epoch 2/10
131/131 [==============================] - 8s 51ms/step - loss: 0.0360 - accuracy: 0.9899 - val_loss: 0.0463 - val_accuracy: 0.9854
Epoch 3/10
131/131 [==============================] - 8s 55ms/step - loss: 0.0122 - accuracy: 0.9971 - val_loss: 0.0398 - val_accuracy: 0.9896
Epoch 4/10
131/131 [==============================] - 10s 68ms/step - loss: 0.0054 - accuracy: 0.9988 - val_loss: 0.0733 - val_accuracy: 0.9823
Epoch 5/10
131/131 [==============================] - 9s 60ms/step - loss: 0.0027 - accuracy: 0.9993 - val_loss: 0.0546 - val_accuracy: 0.9896
Epoch 6/10
131/131 [==============================] - 7s 49ms/step - loss: 0.0020 - accuracy: 0.9993 - val_loss: 0.0502 - val_accuracy: 0.9896
Epoch 7/10
131/131 [==============================] - 7s 49ms/step - loss: 9.1944e-04 - accuracy: 0.9998 - val_loss: 0.0514 - val_accuracy: 

### Predicting and testing

In [22]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  pred_text = encoder.encode(pred_text) # encoding the text to be predicted
  pred_text = tf.cast(pred_text, tf.float32) # Casts a tensor to a float32
  prediction = model.predict(tf.expand_dims(pred_text, 0)).tolist()
  if prediction[0][0] < 0.5:
    prediction.append('ham')
  else:
    prediction.append('spam')
  return (prediction)
pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

[[-3.4903531074523926], 'ham']


In [23]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()

You passed the challenge. Great job!
